## Python 缓存机制使用指南

### lru_cache

- maxsize=128
- typed=False

### diskcache

In [31]:
import diskcache

cache = diskcache.Cache("./tmp/demo_cache")
# 实例化缓存，指定大小限制为100MB, 根据LRU算法清理最少使用的缓存项
cache = diskcache.Cache('/tmp/demo_cache', size_limit=1024 * 1024 * 1024)
cache = diskcache.Cache('/tmp/demo_cache', size_limit=1 << 40)  # 1T
cache = diskcache.Cache('/tmp/demo_cache', size_limit=1 << 30)  # 1G
cache = diskcache.Cache('/tmp/demo_cache', size_limit=1 << 20)  # 1M

cache.set('key1', 'value1')
cache.set('key2', 'value2')
cache["key3"] = "value3"

# 删除指定的缓存项
cache.delete('key1')

print(cache.get('key1'))
print(cache.get('key2'))
print(cache.get('key3'))

print("key1" in cache)
print("key2" in cache)
print("key3" in cache)

None
value2
value3
False
True
True


In [32]:
# 使用lru_cache装饰器自动缓存函数结果
@cache.memoize()
def compute_expensive_operation(x):
    # 模拟耗时计算
    return x * x * x

import time
start_time = time.time()
print(compute_expensive_operation(3))  # 输出: 27
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

# 第二次调用，直接从缓存获取结果
start_time = time.time()
print(compute_expensive_operation(3))  # 输出: 27
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

27
Time taken: 0.001703500747680664 seconds
27
Time taken: 0.00021505355834960938 seconds


In [33]:
stats = cache.stats()

print("Cache hits:", stats[0])
print("Cache misses:", stats[1])

Cache hits: 9
Cache misses: 1


In [37]:
for key in cache:
    print(key, cache[key])

b'key' value
key3 value3
('__main__.compute_expensive_operation', 3, None) 27
key2 value2


In [39]:
print(cache.peekitem())  # 看看最后一个
print(cache.peekitem(last=False))  # 看看第一个

('key2', 'value2')
(b'key', 'value')


In [28]:
result = cache.pop('key2', expire_time=True, tag=True)
result

('value2', None, None)

In [ ]:
cache.expire()  # 清理过期的缓存项

## 多层次缓存管理

In [ ]:
import diskcache as dc

# 创建内存缓存和磁盘缓存
memory_cache = dc.Cache('./tmp/demo_cache_memory')
disk_cache = dc.Cache('./tmp/demo_cache_disk')

# 创建 FanoutCache，传入多个缓存源
fanout_cache = dc.FanoutCache([memory_cache, disk_cache])

# 向缓存写入数据
fanout_cache['user_123'] = {'name': 'Alice', 'age': 30}

# 从缓存读取数据
user = fanout_cache['user_123']
print(user)  # 输出：{'name': 'Alice', 'age': 30}